In [214]:
#PROBABILITY DETECTION MODEL


#create script to subsample the detection results at a desired FPS and a dataframe of frames and detections

#the script should be ran across .5,1,2,3,4,5 FPS (where the video is >=5 fps)

#input will be a video stream (a stream of images effectively), and will at the maximum the number of frames will be 5x the length of the video in seconds
import cv2
import numpy as np
import pandas as pd
import boto3
from boto3.dynamodb.conditions import Key, Attr
import joblib
import re
import math
import ast
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

In [215]:
#is the expectation for this tool to be used with real time or just a stream of images?

In [216]:

#loading model
dynamodb = boto3.client('dynamodb')
table_name = 'robomladen_frame_metadata'
response = dynamodb.scan(
    TableName=table_name
)

# Handle the scan results
list_of_items = []
for item in response['Items']:
    #print(item)
    list_of_items.append(item)
#print(list_of_items[1])

list_df = pd.DataFrame(list_of_items)

print(list_df)
#for i in range(1,len(list_df)-1):
    #print(list_df[i]['timestamp'])



    preprocessing_step              model        camera_id         frame_id  \
0       {'BOOL': True}  {'S': 'intruder'}       {'N': '1'}    {'S': '0001'}   
1       {'BOOL': True}                NaN       {'N': '1'}    {'S': '0001'}   
2       {'BOOL': True}  {'S': 'intruder'}       {'N': '1'}    {'S': '0002'}   
3       {'BOOL': True}  {'S': 'intruder'}       {'N': '1'}    {'S': '0003'}   
4       {'BOOL': True}    {'S': 'weapon'}       {'N': '1'}    {'S': '0003'}   
..                 ...                ...              ...              ...   
624     {'BOOL': True}  {'S': 'intruder'}  {'S': '076098'}  {'S': '000101'}   
625     {'BOOL': True}  {'S': 'fpfinder'}  {'S': '076098'}  {'S': '000101'}   
626     {'BOOL': True}  {'S': 'fpfinder'}  {'S': '076098'}  {'S': '000101'}   
627     {'BOOL': True}    {'S': 'weapon'}  {'S': '076098'}  {'S': '000101'}   
628     {'BOOL': True}    {'S': 'weapon'}  {'S': '076098'}  {'S': '000101'}   

         preprocessing_option  \
0          {'S': '

In [217]:

joblib.dump(list_of_items, "model_results.pkl" , compress=0, protocol=None, cache_size=None)

['model_results.pkl']

In [218]:
model_results = joblib.load("model_results.pkl")
for i in range(0,len(model_results)-1):
    model_results[i]['simple_time'] = round(float(model_results[i]['timestamp']['N']),2)

In [219]:
model_results_df = pd.DataFrame(model_results)

#hard coded, adjust as needed later
df = model_results_df.loc[model_results_df['batch_id'] != "6ecf25d7-3564-4316-903e-7a3508c10cc9"]
df = df.sort_values(by = ['simple_time'])



In [220]:

columns = ['frame_id','simple_time','detections']
print(df[columns])


            frame_id   simple_time  \
1      {'S': '0001'}  1.692732e+09   
2      {'S': '0002'}  1.692732e+09   
0      {'S': '0001'}  1.692732e+09   
3      {'S': '0003'}  1.692732e+09   
4      {'S': '0003'}  1.692732e+09   
..               ...           ...   
19   {'S': '000008'}  1.694545e+09   
20   {'S': '000008'}  1.694545e+09   
21   {'S': '000009'}  1.694545e+09   
22   {'S': '000009'}  1.694545e+09   
628  {'S': '000101'}           NaN   

                                            detections  
1    {'L': [{'S': '[["person", 0.92482590675354, [1...  
2    {'L': [{'S': '[["person", 0.76482590675354, [1...  
0                                            {'L': []}  
3    {'L': [{'S': '[["person", 0.3018590675354, [13...  
4    {'L': [{'S': '[["gun", 0.33333, [1300, 150, 40...  
..                                                 ...  
19                                           {'L': []}  
20                                           {'L': []}  
21                            

In [221]:
#assuming 30 fps camera
#1800 frames per minute
#.5 fps = frame every 2 seconds, 30 frames per minute
#1 fps = frame every 1 minute, 60 frames per minute 
#2 fps = frame every 30 seconds, 120 frames per minute
#3 fps = frame every 20 seconds, 180 frames per minute
#4 fps = frame every 15 seconds, 240 frames per minute
#5 fps = frame every 12 seconds, 300 frames per minute 

fps = [.5,1,2,3,4,5]
camera_intervals = [10]


In [222]:

#the input here is one output of one frame, indexes on "detections"
def detections_string_fix(individual_list_of_detections):
    
    fixed_list = []
    for p in range(0,len(individual_list_of_detections)):
        y = individual_list_of_detections[p][2:-2].split(',')
        y[0] = y[0][1:-1]
        y[1] = float(y[1])
        y = y[0:2]
        fixed_list.append(y)
    return fixed_list

In [223]:
def get_detections_of_frame(query_results):
    
    detections_list = []
    for i in range(len(query_results)):
    #for i in range(15,30):
        
        frame_detections = {
            "frame_id" : 0,
            "number_detections": 0,
            "detections" : [],
            "max_confidence" : 0
        }
        frame_detections["frame_id"] = int(query_results[i]['frame_id']['S'])
        if len((query_results[i]['detections']['L'])) > 0:
            #print("Detections" , len((model_results[i]['detections']['L'])), model_results[i]['detections']['L'] )
            
            for p in range(0, len((query_results[i]['detections']['L']))):
                frame_detections["detections"].append(query_results[i]['detections']['L'][p]['S'])
                #frame_detections["detections"].append(((model_results[i]['detections']['L'][i]))
        
        
        #determining the maximium confidence from a frame
        
        
        frame_detections['number_detections'] = len(frame_detections['detections'])
        
        #frame_detections['detections'] = detections_string_fix(frame_detections['detections'])
        detections_list.append(frame_detections)
        
    
    return detections_list
detections_per_frame = get_detections_of_frame(model_results)



print(len(detections_per_frame))
for i in range(0,len(detections_per_frame)-1):
    detections_per_frame[i]["detections"] = detections_string_fix(detections_per_frame[i]["detections"])


629


In [231]:
data = detections_per_frame



In [232]:
print(data)

[{'frame_id': 1, 'number_detections': 0, 'detections': [], 'max_confidence': 0}, {'frame_id': 1, 'number_detections': 1, 'detections': [['person', 0.92482590675354]], 'max_confidence': 0}, {'frame_id': 2, 'number_detections': 1, 'detections': [['person', 0.76482590675354]], 'max_confidence': 0}, {'frame_id': 3, 'number_detections': 1, 'detections': [['person', 0.3018590675354]], 'max_confidence': 0}, {'frame_id': 3, 'number_detections': 1, 'detections': [['gun', 0.33333]], 'max_confidence': 0}, {'frame_id': 1, 'number_detections': 1, 'detections': [['person', 0.621976375579834]], 'max_confidence': 0}, {'frame_id': 1, 'number_detections': 1, 'detections': [['person', 0.621976375579834]], 'max_confidence': 0}, {'frame_id': 2, 'number_detections': 1, 'detections': [['person', 0.8111607432365417]], 'max_confidence': 0}, {'frame_id': 2, 'number_detections': 1, 'detections': [['person', 0.8111607432365417]], 'max_confidence': 0}, {'frame_id': 3, 'number_detections': 1, 'detections': [['perso

In [238]:
import pandas as pd

def indexes_and_pds(data):
    # Initialize an empty list to store the results
    results = []

    # Loop through different confidence thresholds (from 0.01 to 1.0)
    for i in range(1, 101):
        confidence_threshold = i * 0.01
        
        # Loop through different frame rates (0.5 FPS, 1 FPS, 2 FPS, 3 FPS, 4 FPS, and 5 FPS)
        for fps in [0.5, 1, 2, 3, 4, 5]:
            # Calculate the frame interval for the current FPS
            frame_interval = int(30 / fps)  # Assuming a 30 FPS video

            # Initialize a dictionary to store results for the current combination
            result_dict = {
                'Confidence': confidence_threshold,
                'FPS': fps
            }

            true_positives = 0
            total_detections = 0

            # Iterate through frames starting from index 0
            current_index = 0
            while current_index < len(data):
                # Extract a frame
                frame_data = data[current_index]

                # Check if there are detections
                number_detections = frame_data['number_detections']
                if number_detections > 0:
                    detections = frame_data['detections']
                    for detection in detections:
                        if detection[0] == 'person' and detection[1] >= confidence_threshold:
                            true_positives += 1

                    total_detections += number_detections

                current_index += frame_interval  # Move to the next frame based on the frame interval

            # Calculate Pd for the current combination
            if total_detections == 0:
                pd_value = 0  # Handle the case when there are no detections for this combination
            else:
                pd_value = true_positives / total_detections

            # Add Pd value to the result dictionary
            result_dict['Pd'] = pd_value

            # Append the result dictionary to the results list
            results.append(result_dict)

    # Create a DataFrame from the results list
    result_df = pd.DataFrame(results)

    return result_df  # Return the DataFrame

# Call the function with your data
result_df = indexes_and_pds(data)

# Print the resulting DataFrame
print(result_df)


     Confidence  FPS   Pd
0          0.01  0.5  1.0
1          0.01  1.0  1.0
2          0.01  2.0  1.0
3          0.01  3.0  1.0
4          0.01  4.0  1.0
..          ...  ...  ...
595        1.00  1.0  0.0
596        1.00  2.0  0.0
597        1.00  3.0  0.0
598        1.00  4.0  0.0
599        1.00  5.0  0.0

[600 rows x 3 columns]


In [239]:
df = indexes_and_pds(data)
print(df)

     Confidence  FPS   Pd
0          0.01  0.5  1.0
1          0.01  1.0  1.0
2          0.01  2.0  1.0
3          0.01  3.0  1.0
4          0.01  4.0  1.0
..          ...  ...  ...
595        1.00  1.0  0.0
596        1.00  2.0  0.0
597        1.00  3.0  0.0
598        1.00  4.0  0.0
599        1.00  5.0  0.0

[600 rows x 3 columns]


In [241]:

# Create an interactive heatmap with hover text using Plotly
heatmap = px.imshow(
    df.pivot_table(index="FPS", columns="Confidence", values="Pd"),
    x=df['Confidence'].unique(),
    y=df['FPS'].unique(),
    color_continuous_scale="viridis",
    labels={"color": "Probability of Detection (Pd)"},
    title="Probability of Detection Heatmap",
    aspect = "auto",
)

heatmap.update_traces(text=df.pivot_table(index="FPS", columns="Confidence", values="Pd").values.tolist(), showscale=True)

# Adjust the size of the heatmap trace
heatmap.update_layout(
    autosize=False,  # Disable automatic resizing
    width=900,  # Set the desired width
    height=650,  # Set the desired height
)
